## 📈 Stock Predictor - Machine Learning Model
This notebook trains a machine learning model to predict the next-day closing price using technical indicators.

In [ ]:
# Parameters
ticker = "AAPL"

In [ ]:
! pip install sklearn

In [ ]:
import pandas as pd
from src.data.fetcher import fetch_data
from src.features.indicators import (
    add_moving_average, add_rsi, add_macd, add_bollinger_bands, add_ema_crossover
)
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import joblib

In [ ]:
df = fetch_data(ticker, period="1y")
df = add_moving_average(df)
df = add_bollinger_bands(df)
df = add_rsi(df)
df = add_macd(df)
df = add_ema_crossover(df)

df.dropna(inplace=True)
df["Target"] = df["Close"].shift(-1)
df.dropna(inplace=True)

In [ ]:
features = [
    "MA_20", "BB_Upper", "BB_Lower", "RSI",
    "MACD", "MACD_Signal", "EMA_9", "EMA_21"
]
X = df[features]
y = df["Target"]

X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=False, test_size=0.2)

In [ ]:
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

preds = model.predict(X_test)
rmse = mean_squared_error(y_test, preds, squared=False)
print(f"Test RMSE: {rmse:.2f}")

In [ ]:
joblib.dump(model, f"../src/model/trained_model_{ticker}.pkl")
print(f"✅ Model saved as trained_model_{ticker}.pkl")